In [45]:
import numpy as np
import os
import matplotlib.pyplot as plt

In [46]:
joint_training_paste = [0.411] * 10

finetuning = [
	[0.515], 
	[0.102, 0.778], 
	[0.072, 0.110, 0.524], 
	[0.152, 0.181, 0.056, 0.484], 
	[0.158, 0.119, 0.060, 0.052, 0.784], 
	[0.174, 0.180, 0.108, 0.109, 0.383, 0.704],
	[0.132, 0.126, 0.056, 0.025, 0.449, 0.215, 0.615],
	[0.076, 0.116, 0.056, 0.025, 0.400, 0.465, 0.100, 0.484],
	[0.108, 0.164, 0.059, 0.058, 0.337, 0.123, 0.119, 0.120, 0.512],
	[0.073, 0.180, 0.081, 0.099, 0.210, 0.199, 0.198, 0.093, 0.085, 0.163],
]

replay_paste_mem100 = [
	[0.560], 
	[0.462, 0.755], 
	[0.360, 0.748, 0.339],
	[0.410, 0.745, 0.297, 0.453],
	[0.330, 0.725, 0.273, 0.417, 0.689],
	[0.304, 0.738, 0.266, 0.420, 0.672, 0.578],
	[0.314, 0.725, 0.235, 0.421, 0.622, 0.496, 0.559],
	[0.244, 0.669, 0.227, 0.360, 0.616, 0.468, 0.461, 0.444],
	[0.347, 0.693, 0.238, 0.402, 0.634, 0.470, 0.486, 0.353, 0.449],
	[0.315, 0.693, 0.208, 0.395, 0.610, 0.495, 0.451, 0.356, 0.368, 0.080]
]

replay_stfpm_mem100 = [
	[0.516],
	[0.418, 0.796],
	[0.317, 0.750, 0.287],
	[0.258, 0.707, 0.247, 0.395],
	[0.118, 0.628, 0.146, 0.232, 0.505],
	[0.260, 0.722, 0.231, 0.342, 0.589, 0.616],
	[0.165, 0.696, 0.184, 0.228, 0.529, 0.411, 0.485],
	[0.188, 0.664, 0.194, 0.235, 0.557, 0.535, 0.574, 0.350],
	[0.187, 0.707, 0.189, 0.309, 0.571, 0.468, 0.598, 0.324, 0.411],
	[0.186, 0.680, 0.197, 0.312, 0.451, 0.514, 0.562, 0.330, 0.281, 0.105],
]

compressed_replay_paste_mem100 = [
	[0.564], 
	[0.487, 0.763],
	[0.467, 0.728, 0.336],
	[0.428, 0.725, 0.296, 0.452],
	[0.395, 0.732, 0.274, 0.417, 0.699],
	[0.391, 0.714, 0.247, 0.419, 0.633, 0.592],
	[0.324, 0.702, 0.240, 0.435, 0.686, 0.523, 0.558],
	[0.264, 0.701, 0.216, 0.399, 0.630, 0.526, 0.522, 0.440],
	[0.297, 0.726, 0.229, 0.423, 0.660, 0.508, 0.536, 0.367, 0.443],
	[0.299, 0.702, 0.234, 0.379, 0.571, 0.490, 0.506, 0.370, 0.391, 0.095],
]

quantized_compressed_replay_paste_mem100 = [
	[0.609],
	[0.557, 0.674],
	[0.554, 0.635, 0.255],
	[0.562, 0.623, 0.216, 0.406],
	[0.526, 0.574, 0.191, 0.374, 0.635],
	[0.560, 0.562, 0.188, 0.361, 0.597, 0.470],
	[0.562, 0.549, 0.180, 0.356, 0.576, 0.403, 0.464],
	[0.537, 0.533, 0.158, 0.290, 0.544, 0.375, 0.432, 0.304],
	[0.556, 0.532, 0.168, 0.346, 0.541, 0.349, 0.425, 0.241, 0.282],
	[0.548, 0.503, 0.154, 0.253, 0.504, 0.331, 0.408, 0.200, 0.163, 0.139]
]

pca_compression_compressed_replay_paste_mem100_variance098 = [
	[0.612],
	[0.556, 0.675],
	[0.558, 0.628, 0.246],
	[0.557, 0.611, 0.208, 0.372],
	[0.534, 0.546, 0.179, 0.341, 0.636],
	[0.551, 0.551, 0.185, 0.318, 0.568, 0.472],
	[0.519, 0.546, 0.170, 0.298, 0.578, 0.427, 0.454],
	[0.491, 0.509, 0.157, 0.273, 0.572, 0.411, 0.405, 0.304],
	[0.505, 0.513, 0.158, 0.304, 0.553, 0.317, 0.391, 0.242, 0.323],
	[0.541, 0.467, 0.152, 0.280, 0.489, 0.290, 0.352, 0.217, 0.199, 0.099],
]


In [47]:
def prepare_data(data):
    """
    Prepares learning curve data by calculating the average F1 score of seen tasks at each step.
    
    Args:
        data (list of lists): Each sublist contains F1 scores for seen tasks at that step
        
    Returns:
        list: Average F1 scores for each step (10 values for 10 tasks)
    """
    averages = []
    
    for task_results in data:
        avg = sum(task_results) / len(task_results)
        averages.append(round(avg, 3))
        
    return averages

In [48]:
num_tasks = 10
tasks = np.arange(num_tasks)

In [49]:
plt.figure(figsize=(10, 6))

plt.plot(tasks, prepare_data(compressed_replay_paste_mem100), '-', label='Compressed-Replay-100 PASTE', color='green')
plt.plot(tasks, prepare_data(replay_stfpm_mem100), '-', label='Replay-100 STFPM', color='orange')
plt.plot(tasks, prepare_data(replay_paste_mem100), '-', label='Replay-100 PASTE', color='blue')
plt.plot(tasks, joint_training_paste, '-', label='Joint Training PASTE', color='red')
plt.plot(tasks, prepare_data(finetuning), '-', label='Fine-Tuning PASTE', color='pink')

plt.grid(True)
plt.xlabel('Task')
plt.ylabel('F1 pixel-level')
plt.title('F1 pixel-level results')
plt.legend()

plt.grid(True, which='minor', linestyle='--', alpha=0.4)
plt.grid(True, which='major', linestyle='-', alpha=0.6)

plt.tight_layout()

layer_dir = os.path.join('plots', 'f1_pixel_level')
os.makedirs(layer_dir, exist_ok=True)
plot_filename = os.path.join(layer_dir, 'f1_pixel_level.png')
plt.savefig(plot_filename)

plt.close()

In [50]:
plt.figure(figsize=(10, 6))

plt.plot(tasks, prepare_data(pca_compression_compressed_replay_paste_mem100_variance098), '-', label='PCA-Compression-Compressed-Replay-100 PASTE', color='blue')
plt.plot(tasks, prepare_data(quantized_compressed_replay_paste_mem100), '-', label='Quantized-Compressed-Replay-100 PASTE', color='orange')
plt.plot(tasks, prepare_data(compressed_replay_paste_mem100), '-', label='Compressed-Replay-100 PASTE', color='green')
plt.plot(tasks, joint_training_paste, '-', label='Joint Training PASTE', color='red')
plt.plot(tasks, prepare_data(finetuning), '-', label='Fine-Tuning PASTE', color='pink')

plt.grid(True)
plt.xlabel('Task')
plt.ylabel('F1 pixel-level')
plt.title('F1 pixel-level results')
plt.legend()

plt.grid(True, which='minor', linestyle='--', alpha=0.4)
plt.grid(True, which='major', linestyle='-', alpha=0.6)

plt.tight_layout()

layer_dir = os.path.join('plots', 'f1_pixel_level')
os.makedirs(layer_dir, exist_ok=True)
plot_filename = os.path.join(layer_dir, 'f1_pixel_level_compression.png')
plt.savefig(plot_filename)

plt.close()